##  Import Libraries

In [ ]:
import dotenv
import os
import deepl
import pyperclip

dotenv.load_dotenv()

True

## Test API Key

In [12]:
DeepL_API_KEY = os.getenv("DeepL_API_KEY")
#print(DeepL_API_KEY)

In [13]:
translator = deepl.Translator(DeepL_API_KEY)
def translate_text_from_ID_to_EN(text):
    result = translator.translate_text(text, source_lang="ID", target_lang="EN-US")
    return result.text

In [14]:
text = input("Masukkan teks dalam bahasa Indonesia: ")
translated_text = translate_text_from_ID_to_EN(text)
print("Teks dalam bahasa Inggris:\n", translated_text)

Teks dalam bahasa Inggris:
 hello my name is DIrzy. I love computer programming


## Citation Checker v-1

In [ ]:
# def translate_text_deepl(api_key, text, target_lang="EN"):
#     """
#     Mengirimkan request ke DeepL API untuk menerjemahkan teks.
#     """
#     url = "https://api-free.deepl.com/v2/translate"
#     payload = {
#         "auth_key": api_key,
#         "text": text,
#         "target_lang": target_lang
#     }
#     response = requests.post(url, data=payload)
#     response_json = response.json()
#     if "translations" in response_json:
#         return response_json["translations"][0]["text"]
#     else:
#         print("Error dalam menerjemahkan teks:", response_json)
#         return None

### Sentence Based

In [33]:
import os
import deepl
import PyPDF2
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk

# Pastikan paket NLTK yang dibutuhkan sudah ada (tokenizer)
nltk.download('punkt')
nltk.download('punkt_tab')


# Ganti dengan API key DeepL gratis Anda
DeepL_API_KEY = os.getenv("DeepL_API_KEY")  # Ubah dengan API key milik Anda

# Inisialisasi translator dari library DeepL
translator = deepl.Translator(DeepL_API_KEY)

def translate_text_from_ID_to_EN(text):
    """
    Menerjemahkan teks dari Bahasa Indonesia ke Bahasa Inggris (EN-US)
    menggunakan DeepL API melalui library DeepL.
    """
    result = translator.translate_text(text, source_lang="ID", target_lang="EN-US")
    return result.text

def extract_text_by_page(pdf_path):
    """
    Mengekstrak teks dari setiap halaman file PDF dan mengembalikannya
    sebagai dictionary dengan nomor halaman sebagai key.
    """
    pages_text = {}
    try:
        with open(pdf_path, "rb") as f:
            pdf_reader = PyPDF2.PdfReader(f)
            for page_num, page in enumerate(pdf_reader.pages):
                page_text = page.extract_text()
                pages_text[page_num + 1] = page_text
        return pages_text
    except Exception as e:
        print("Error membaca file PDF:", e)
        return {}

def find_sentence_matches(citation_text, pages_text, similarity_threshold=0.6):
    """
    Memecah teks PDF per halaman menjadi kalimat dan menghitung cosine similarity
    antara teks sitasi (hasil terjemahan) dengan tiap kalimat PDF.
    Mengembalikan daftar tuple yang berisi (halaman, kalimat, nilai kemiripan).
    """
    matches = []
    for page, text in pages_text.items():
        if not text:
            continue
        # Pecah teks per halaman menjadi kalimat
        sentences = nltk.tokenize.sent_tokenize(text)
        for sentence in sentences:
            # Buat corpus untuk perbandingan: [teks sitasi, kalimat PDF]
            corpus = [citation_text, sentence]
            vectorizer = TfidfVectorizer().fit(corpus)
            vectors = vectorizer.transform(corpus)
            cos_sim = cosine_similarity(vectors[0], vectors[1])[0][0]
            if cos_sim >= similarity_threshold:
                matches.append((page, sentence, cos_sim))
    return matches

def main():
    # 1. Input teks sitasi Bahasa Indonesia dari Word
    citation_indonesian = input("Masukkan teks sitasi (Bahasa Indonesia): ")

    # 2. Terjemahkan teks tersbut ke Bahasa Inggris dengan DeepL API
    citation_translated = translate_text_from_ID_to_EN(citation_indonesian)
    if citation_translated:
        print("\nHasil back-translation:", citation_translated)
    else:
        print("Gagal menerjemahkan teks. Periksa API key dan koneksi internet Anda.")
        return

    # 3. Input path file PDF (paper Bahasa Inggris)
    pdf_path = input("\nMasukkan path file PDF referensi: ")
    if not os.path.exists(pdf_path):
        print("File tidak ditemukan di path yang diberikan.")
        return

    pages_text = extract_text_by_page(pdf_path)
    if not pages_text:
        print("Tidak dapat mengekstrak teks dari file PDF.")
        return

    # 4. Minta threshold kemiripan (contoh, 0.6 sebagai default)
    try:
        threshold = float(input("\nMasukkan threshold kemiripan (contoh: 0.6): "))
    except ValueError:
        threshold = 0.3
        print("Input tidak valid. Menggunakan threshold default 0.6")

    # 5. Cari kemiripan di tiap kalimat dalam file PDF
    matches = find_sentence_matches(citation_translated, pages_text, similarity_threshold=threshold)

    # 6. Tampilkan hasil pencarian
    if matches:
        print("\nDitemukan kemiripan di kalimat-kalimat berikut:")
        for page, sentence, sim in matches:
            print(f"Halaman {page}, Kemiripan: {sim:.2f}")
            print("Kalimat:", sentence)
            print("-" * 40)
    else:
        print("\nTidak ditemukan kemiripan yang signifikan dalam file PDF.")

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to C:\Users\Dirzy
[nltk_data]     Adam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Dirzy
[nltk_data]     Adam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!



Hasil back-translation: In general, conventional methods face a dilemma between model complexity, estimation accuracy, data requirements and generalizability.

Tidak ditemukan kemiripan yang signifikan dalam file PDF.


### Pragraph Based

In [34]:
import os
import deepl
import PyPDF2
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk

# Pastikan resource yang diperlukan sudah diunduh
nltk.download('punkt')

# Ganti dengan API key DeepL gratis Anda
DeepL_API_KEY = os.getenv("DeepL_API_KEY")  # Ubah dengan API key milik Anda

# Inisialisasi translator dari library DeepL
translator = deepl.Translator(DeepL_API_KEY)

def translate_text_from_ID_to_EN(text):
    """
    Menerjemahkan teks dari Bahasa Indonesia ke Bahasa Inggris (EN-US)
    menggunakan DeepL API melalui library DeepL.
    """
    result = translator.translate_text(text, source_lang="ID", target_lang="EN-US")
    return result.text

def extract_text_by_page(pdf_path):
    """
    Mengekstrak teks dari setiap halaman file PDF dan mengembalikannya
    sebagai dictionary dengan nomor halaman sebagai key.
    """
    pages_text = {}
    try:
        with open(pdf_path, "rb") as f:
            pdf_reader = PyPDF2.PdfReader(f)
            for page_num, page in enumerate(pdf_reader.pages):
                page_text = page.extract_text()
                pages_text[page_num + 1] = page_text
        return pages_text
    except Exception as e:
        print("Error membaca file PDF:", e)
        return {}

def find_sentence_matches(citation_text, pages_text, similarity_threshold=0.6):
    """
    Membagi teks per halaman menjadi kalimat dan menghitung cosine similarity
    antara teks sitasi (hasil terjemahan) dengan tiap kalimat PDF.
    """
    matches = []
    for page, text in pages_text.items():
        if not text:
            continue
        # Memecah teks pada halaman menjadi kalimat
        sentences = nltk.tokenize.sent_tokenize(text)
        for sentence in sentences:
            corpus = [citation_text, sentence]
            vectorizer = TfidfVectorizer().fit(corpus)
            vectors = vectorizer.transform(corpus)
            cos_sim = cosine_similarity(vectors[0], vectors[1])[0][0]
            if cos_sim >= similarity_threshold:
                matches.append((page, sentence, cos_sim))
    return matches

def find_paragraph_matches(citation_text, pages_text, similarity_threshold=0.6):
    """
    Membagi teks per halaman menjadi paragraf (dengan pemisah dua baris baru)
    dan menghitung cosine similarity antara teks sitasi (hasil terjemahan)
    dengan tiap paragraf PDF.
    """
    matches = []
    for page, text in pages_text.items():
        if not text:
            continue
        # Memecah teks menjadi paragraf: asumsikan paragraf dipisahkan oleh "\n\n"
        paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]
        for paragraph in paragraphs:
            corpus = [citation_text, paragraph]
            vectorizer = TfidfVectorizer().fit(corpus)
            vectors = vectorizer.transform(corpus)
            cos_sim = cosine_similarity(vectors[0], vectors[1])[0][0]
            if cos_sim >= similarity_threshold:
                matches.append((page, paragraph, cos_sim))
    return matches

def main():
    # 1. Input teks sitasi Bahasa Indonesia dari Word
    citation_indonesian = input("Masukkan teks sitasi (Bahasa Indonesia): ")

    # 2. Terjemahkan teks menuju Bahasa Inggris menggunakan DeepL API
    citation_translated = translate_text_from_ID_to_EN(citation_indonesian)
    if citation_translated:
        print("\nHasil back-translation:", citation_translated)
    else:
        print("Gagal menerjemahkan teks. Periksa API key dan koneksi internet Anda.")
        return

    # 3. Input path file PDF (paper Bahasa Inggris)
    pdf_path = input("\nMasukkan path file PDF referensi: ")
    if not os.path.exists(pdf_path):
        print("File tidak ditemukan di path yang diberikan.")
        return

    pages_text = extract_text_by_page(pdf_path)
    if not pages_text:
        print("Tidak dapat mengekstrak teks dari file PDF.")
        return

    # 4. Minta threshold kemiripan (misalnya 0.6 sebagai default)
    try:
        threshold = float(input("\nMasukkan threshold kemiripan (contoh: 0.6): "))
    except ValueError:
        threshold = 0.6
        print("Input tidak valid. Menggunakan threshold default 0.6")

    # 5. Pilih mode pemeriksaan: Kalimat atau Paragraf
    mode = input("\nPilih mode pemeriksaan: ketik 's' untuk kalimat, 'p' untuk paragraf: ").lower()
    if mode == 'p':
        matches = find_paragraph_matches(citation_translated, pages_text, similarity_threshold=threshold)
        tipe_cek = "Paragraf"
    else:
        matches = find_sentence_matches(citation_translated, pages_text, similarity_threshold=threshold)
        tipe_cek = "Kalimat"

    # 6. Tampilkan hasil pencarian
    if matches:
        print(f"\nDitemukan kemiripan pada {tipe_cek} berikut:")
        for page, text_match, sim in matches:
            print(f"Halaman {page}, Kemiripan: {sim:.2f}")
            print("Teks:\n", text_match)
            print("-" * 40)
    else:
        print(f"\nTidak ditemukan kemiripan yang signifikan pada pencarian {tipe_cek}.")

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to C:\Users\Dirzy
[nltk_data]     Adam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Hasil back-translation: In general, conventional methods face a dilemma between model complexity, estimation accuracy, data requirements and generalizability.

Ditemukan kemiripan pada Paragraf berikut:
Halaman 1, Kemiripan: 0.13
Teks:
 Estimation of Cyclable Lithium for Li-ion Battery
State-of-Health Monitoring
Saehong Park, Dong Zhang, Reinhardt Klein, Scott Moura
Abstract —State of health (SOH) estimation for Li-ion bat-
teries enables high-ﬁdelity monitoring and high-performance in
advanced battery management systems for applications such as
mobile devices, electriﬁed transportation, and energy storage. In
order to achieve accurate SOH information, this paper improves
the output voltage prediction by considering state-dependent
parameters for a reduced-order electrochemical model. In ad-
dition, the SOH information is deﬁned as the total moles of
lithium in both solid-phase and electrolyte-phase, which directly
affects the initial conditions of the electrochemical states in the
mo

## Citation Checker v-2

In [35]:
import os
import deepl
import PyPDF2
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from sentence_transformers import SentenceTransformer, util

# Pastikan resource NLTK telah diunduh
nltk.download('punkt')

# Ganti dengan API key DeepL gratis Anda (misalnya melalui environment variable)
DeepL_API_KEY = os.getenv("DeepL_API_KEY")  # Ubah dengan API key milik Anda

# Inisialisasi translator dari library DeepL
translator = deepl.Translator(DeepL_API_KEY)

# Inisialisasi model untuk representasi semantik (Sentence-BERT)
semantic_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

def translate_text_from_ID_to_EN(text):
    """
    Menerjemahkan teks dari Bahasa Indonesia ke Bahasa Inggris (EN-US)
    menggunakan DeepL API melalui library DeepL.
    """
    result = translator.translate_text(text, source_lang="ID", target_lang="EN-US")
    return result.text

def extract_text_by_page(pdf_path):
    """
    Mengekstrak teks dari setiap halaman file PDF dan mengembalikannya
    sebagai dictionary dengan nomor halaman sebagai key.
    """
    pages_text = {}
    try:
        with open(pdf_path, "rb") as f:
            pdf_reader = PyPDF2.PdfReader(f)
            for page_num, page in enumerate(pdf_reader.pages):
                page_text = page.extract_text()
                pages_text[page_num + 1] = page_text
        return pages_text
    except Exception as e:
        print("Error membaca file PDF:", e)
        return {}

def compute_semantic_similarity(text1, text2):
    """
    Menghitung cosine similarity antara dua teks menggunakan
    model embedding dari Sentence-BERT.
    """
    embedding1 = semantic_model.encode(text1, convert_to_tensor=True)
    embedding2 = semantic_model.encode(text2, convert_to_tensor=True)
    return util.pytorch_cos_sim(embedding1, embedding2).item()

def find_sentence_matches(citation_text, pages_text, similarity_threshold=0.6, method="tfidf"):
    """
    Membagi teks per halaman menjadi kalimat dan menghitung cosine similarity
    antara teks sitasi (hasil terjemahan) dengan tiap kalimat PDF.
    
    Parameter:
      - method: "tfidf" (default) atau "semantic". Jika "semantic",
                perhitungan dilakukan menggunakan representasi embedding.
    """
    matches = []
    for page, text in pages_text.items():
        if not text:
            continue
        # Pecah teks menjadi kalimat
        sentences = nltk.tokenize.sent_tokenize(text)
        for sentence in sentences:
            if method == "semantic":
                cos_sim = compute_semantic_similarity(citation_text, sentence)
            else:
                corpus = [citation_text, sentence]
                vectorizer = TfidfVectorizer().fit(corpus)
                vectors = vectorizer.transform(corpus)
                cos_sim = cosine_similarity(vectors[0], vectors[1])[0][0]
            if cos_sim >= similarity_threshold:
                matches.append((page, sentence, cos_sim))
    return matches

def find_paragraph_matches(citation_text, pages_text, similarity_threshold=0.6, method="tfidf"):
    """
    Membagi teks per halaman menjadi paragraf (dengan pemisah dua baris baru)
    dan menghitung cosine similarity antara teks sitasi (hasil terjemahan)
    dengan tiap paragraf PDF.
    
    Parameter:
      - method: "tfidf" (default) atau "semantic". Jika "semantic",
                perhitungan dilakukan menggunakan representasi embedding.
    """
    matches = []
    for page, text in pages_text.items():
        if not text:
            continue
        # Pecah teks menjadi paragraf (asumsi: paragraf dipisahkan oleh "\n\n")
        paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]
        for paragraph in paragraphs:
            if method == "semantic":
                cos_sim = compute_semantic_similarity(citation_text, paragraph)
            else:
                corpus = [citation_text, paragraph]
                vectorizer = TfidfVectorizer().fit(corpus)
                vectors = vectorizer.transform(corpus)
                cos_sim = cosine_similarity(vectors[0], vectors[1])[0][0]
            if cos_sim >= similarity_threshold:
                matches.append((page, paragraph, cos_sim))
    return matches

def main():
    # 1. Input teks sitasi Bahasa Indonesia dari Word
    citation_indonesian = input("Masukkan teks sitasi (Bahasa Indonesia): ")

    # 2. Terjemahkan teks menuju Bahasa Inggris menggunakan DeepL API
    citation_translated = translate_text_from_ID_to_EN(citation_indonesian)
    if citation_translated:
        print("\nHasil back-translation:", citation_translated)
    else:
        print("Gagal menerjemahkan teks. Periksa API key dan koneksi internet Anda.")
        return

    # 3. Input path file PDF (paper Bahasa Inggris)
    pdf_path = input("\nMasukkan path file PDF referensi: ")
    if not os.path.exists(pdf_path):
        print("File tidak ditemukan di path yang diberikan.")
        return

    pages_text = extract_text_by_page(pdf_path)
    if not pages_text:
        print("Tidak dapat mengekstrak teks dari file PDF.")
        return

    # 4. Minta threshold kemiripan (misalnya, 0.6 sebagai default)
    try:
        threshold = float(input("\nMasukkan threshold kemiripan (contoh: 0.6): "))
    except ValueError:
        threshold = 0.6
        print("Input tidak valid. Menggunakan threshold default 0.6")

    # 5. Pilih mode pemeriksaan: Kalimat atau Paragraf
    mode = input("\nPilih mode pemeriksaan: ketik 's' untuk kalimat, 'p' untuk paragraf: ").lower()
    if mode == 'p':
        text_matches_func = find_paragraph_matches
        tipe_cek = "Paragraf"
    else:
        text_matches_func = find_sentence_matches
        tipe_cek = "Kalimat"

    # 6. Pilih metode perbandingan: 1 untuk TF-IDF atau 2 untuk Semantic Representation
    method_choice = input("\nPilih metode perbandingan: ketik '1' untuk TF-IDF, '2' untuk Semantic: ").strip()
    if method_choice == '2':
        method = "semantic"
    else:
        method = "tfidf"

    # 7. Cari kemiripan menggunakan metode dan mode yang dipilih
    matches = text_matches_func(citation_translated, pages_text, similarity_threshold=threshold, method=method)

    # 8. Tampilkan hasil pencarian
    if matches:
        print(f"\nDitemukan kemiripan pada {tipe_cek} berikut (metode: {method}):")
        for page, text_match, sim in matches:
            print(f"Halaman {page}, Kemiripan: {sim:.2f}")
            print("Teks:\n", text_match)
            print("-" * 40)
    else:
        print(f"\nTidak ditemukan kemiripan yang signifikan pada pencarian {tipe_cek} dengan metode {method}.")

if __name__ == "__main__":
    main()

c:\Users\Dirzy Adam\OneDrive\Documents\1_Skripsiku\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to C:\Users\Dirzy
[nltk_data]     Adam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\Dirzy Adam\OneDrive\Documents\1_Skripsiku\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dirzy Adam\.cache\huggingface\hub\models--sentence-transformers--paraphrase-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. Fo


Hasil back-translation: Equivalent circuit models (ECMs) and adaptive filters such as the Kalman Filter (KF) and its variants (EKF, UKF) rely on model accuracy and parameter identification which can be complex and sensitive to operating conditions and measurement noise.

Ditemukan kemiripan pada Kalimat berikut (metode: semantic):
Halaman 1, Kemiripan: 0.33
Teks:
 Estimation of Cyclable Lithium for Li-ion Battery
State-of-Health Monitoring
Saehong Park, Dong Zhang, Reinhardt Klein, Scott Moura
Abstract —State of health (SOH) estimation for Li-ion bat-
teries enables high-ﬁdelity monitoring and high-performance in
advanced battery management systems for applications such as
mobile devices, electriﬁed transportation, and energy storage.
----------------------------------------
Halaman 1, Kemiripan: 0.45
Teks:
 In
order to achieve accurate SOH information, this paper improves
the output voltage prediction by considering state-dependent
parameters for a reduced-order electrochemical model